In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = "https://www.epoolsupply.com"
start_url = f"{base_url}/collections/pool-chlorine-tabs"
headers = {"User-Agent": "Mozilla/5.0"}

all_products = []

# Step 1: Extract category names and links
resp = requests.get(start_url, headers=headers)
soup = BeautifulSoup(resp.text, 'html.parser')

category_data = []

category_blocks = soup.select("div.subcat-grid-item.subcat-type-collection.subcat-shown")
for div in category_blocks:
    a_tag = div.find("a", class_="subcat-grid-link")
    title_tag = div.find("p", class_="subcat-grid-link__title")
    if a_tag and title_tag:
        category_name = title_tag.get_text(strip=True).split('(')[0].strip()
        category_href = a_tag.get("href")
        full_category_url = base_url + category_href
        category_data.append((category_name, full_category_url))

# Step 2 & 3: For each category, extract product links and info
for cat_name, cat_url in category_data:
    print(f"\nCategory: {cat_name}\nURL: {cat_url}")
    try:
        res = requests.get(cat_url, headers=headers)
        cat_soup = BeautifulSoup(res.text, 'html.parser')
        product_tags = cat_soup.select("li.js-pagination-result p.card__title a")

        for tag in product_tags:
            href = tag.get("href")
            if href:
                full_product_link = base_url + href
                print(full_product_link)

                try:
                    detail_resp = requests.get(full_product_link, headers=headers)
                    detail_soup = BeautifulSoup(detail_resp.text, 'html.parser')

                    # Title
                    title_elem = detail_soup.select_one('h1.product-title')
                    title = title_elem.get_text(strip=True) if title_elem else 'N/A'

                    # Price
                    price_main = detail_soup.select_one('div.product-price strong.price__current')
                    price_sup = detail_soup.select_one('div.product-price strong.price__current sup')
                    
                    if price_main:
                        price_text = price_main.get_text(strip=True).replace('USD', '').strip()
                        if price_sup:
                            sup_text = price_sup.get_text(strip=True)
                            price_text = price_text.replace(sup_text, '') + '.' + sup_text
                        price = price_text
                    else:
                        price = 'N/A'


                    # Stock
                    add_btn = detail_soup.select_one('product-form button[name="add"]')
                    stock = 'In stock' if add_btn and 'add to cart' in add_btn.get_text(strip=True).lower() else 'Out of stock'

                    # Description (first p[data-start])
                    desc_elem = detail_soup.select_one('div.product-description p[data-start]')
                    description = desc_elem.get_text(strip=True) if desc_elem else 'N/A'

                    # Overview (strong[data-start] inside p)
                    overview_elements = detail_soup.select('div.product-description p strong[data-start]')
                    overview = ' | '.join(elem.get_text(strip=True) for elem in overview_elements[1:]) if len(overview_elements) > 1 else 'N/A'


                    # Manufacturer SKU
                    sku_elem = detail_soup.select_one('span.product-sku__value')
                    manufacturer_sku = sku_elem.get_text(strip=True) if sku_elem else 'N/A'

                    # Append
                    all_products.append({
                        'Title': title,
                        'Overview': overview,
                        'Price': price,
                        'Stock': stock,
                        'Description': description,
                        'Link': full_product_link,
                        'Manufacturer SKU': manufacturer_sku,
                        'Brand': cat_name
                    })

                    time.sleep(1)

                except Exception as e:
                    print(f"  ⚠ Failed to fetch product info from {full_product_link}: {e}")
    except Exception as e:
        print(f"  ⚠ Failed to process {cat_name} ({cat_url}): {e}")

# DataFrame
df = pd.DataFrame(all_products)
df.head()



Category: Poolife Chlorine & Sanitizers
URL: https://www.epoolsupply.com/collections/poolife-chlorine-sanitizers-tabs
https://www.epoolsupply.com/collections/poolife-chlorine-sanitizers-tabs/products/pool-logic-3-chlorine-tabs-50lbs
https://www.epoolsupply.com/collections/poolife-chlorine-sanitizers-tabs/products/poolife-1-cleaning-tablets
https://www.epoolsupply.com/collections/poolife-chlorine-sanitizers-tabs/products/poolife-bromine-tablets-25-lbs-62182
https://www.epoolsupply.com/collections/poolife-chlorine-sanitizers-tabs/products/poolife-nst-prime-tablets-4-5-lbs
https://www.epoolsupply.com/collections/poolife-chlorine-sanitizers-tabs/products/poolife-nst-prime-tablets-9-lbs-22424
https://www.epoolsupply.com/collections/poolife-chlorine-sanitizers-tabs/products/poolife-nst-prime-tablets-20-2-lbs-22425
https://www.epoolsupply.com/collections/poolife-chlorine-sanitizers-tabs/products/poolife-nst-prime-tablets-39-4-lbs-22426
https://www.epoolsupply.com/collections/poolife-chlorine

,Title,Overview,Price,Stock,Description,Link,Manufacturer SKU,Brand
0,"Poolife 3"" Chlorine Tabs (50lbs) | 42118","steady, consistent chlorination | sanitized, c...",$169.95,In stock,"Poolife 3"" Chlorine Tabs arehighly effective c...",https://www.epoolsupply.com/collections/poolif...,42118,Poolife Chlorine & Sanitizers
1,"Poolife 1"" Cleaning Tablets | 42104 (5LBS)",pool maintenance | 1-inch trichlor tablets | k...,$52.99,In stock,"Poolife 1"" Cleaning Tablets arehighly effectiv...",https://www.epoolsupply.com/collections/poolif...,42104,Poolife Chlorine & Sanitizers
2,Poolife Bromine Tablets | 25 LBS (62182),slow-dissolving tablets | continuous disinfect...,$299.00,Out of stock,Key Features:,https://www.epoolsupply.com/collections/poolif...,62182,Poolife Chlorine & Sanitizers
3,Poolife NST Prime Tablets | 4.5 LBS,chlorine | oxidizers | stabilizers | residenti...,$41.99,Out of stock,Poolife NST Prime Tablets are designed forpool...,https://www.epoolsupply.com/collections/poolif...,22427,Poolife Chlorine & Sanitizers
4,Poolife NST Prime Tablets | 9 LBS (22424),slow-dissolving tablets | chlorine sanitizers ...,$72.99,Out of stock,Poolife NST Prime Tablets in the9 LBSsize are ...,https://www.epoolsupply.com/collections/poolif...,22424,Poolife Chlorine & Sanitizers


In [14]:
df

,Title,Overview,Price,Stock,Description,Link,Manufacturer SKU,Brand
0,"Poolife 3"" Chlorine Tabs (50lbs) | 42118","steady, consistent chlorination | sanitized, c...",$169.95,In stock,"Poolife 3"" Chlorine Tabs arehighly effective c...",https://www.epoolsupply.com/collections/poolif...,42118,Poolife Chlorine & Sanitizers
1,"Poolife 1"" Cleaning Tablets | 42104 (5LBS)",pool maintenance | 1-inch trichlor tablets | k...,$52.99,In stock,"Poolife 1"" Cleaning Tablets arehighly effectiv...",https://www.epoolsupply.com/collections/poolif...,42104,Poolife Chlorine & Sanitizers
2,Poolife Bromine Tablets | 25 LBS (62182),slow-dissolving tablets | continuous disinfect...,$299.00,Out of stock,Key Features:,https://www.epoolsupply.com/collections/poolif...,62182,Poolife Chlorine & Sanitizers
3,Poolife NST Prime Tablets | 4.5 LBS,chlorine | oxidizers | stabilizers | residenti...,$41.99,Out of stock,Poolife NST Prime Tablets are designed forpool...,https://www.epoolsupply.com/collections/poolif...,22427,Poolife Chlorine & Sanitizers
4,Poolife NST Prime Tablets | 9 LBS (22424),slow-dissolving tablets | chlorine sanitizers ...,$72.99,Out of stock,Poolife NST Prime Tablets in the9 LBSsize are ...,https://www.epoolsupply.com/collections/poolif...,22424,Poolife Chlorine & Sanitizers
5,Poolife NST Prime Tablets | 20.2 LBS (22425),larger pools | chlorine | oxidizing | long-las...,$124.99,Out of stock,Poolife NST Prime Tablets in the20.2 LBSsize a...,https://www.epoolsupply.com/collections/poolif...,22425,Poolife Chlorine & Sanitizers
6,Poolife NST Prime Tablets | 39.4 LBS (22426),39.4 LBS | dual-action sanitization | oxidatio...,$276.99,In stock,ThePoolife NST Prime Tabletsin the39.4 LBSsize...,https://www.epoolsupply.com/collections/poolif...,22426,Poolife Chlorine & Sanitizers
7,"Poolife 3"" Cleaning Tablet | 1, 7 OZ (42130)",sanitized and crystal clear | slow-dissolving ...,$6.99,In stock,"Poolife 3"" Cleaning Tablets providelong-lastin...",https://www.epoolsupply.com/collections/poolif...,42130,Poolife Chlorine & Sanitizers
8,"Poolife 3"" Cleaning Tablets | 4.81 LBS (42107)",sanitized and crystal clear | slow-dissolving ...,$42.99,Out of stock,"Poolife 3"" Cleaning Tablets providelong-lastin...",https://www.epoolsupply.com/collections/poolif...,42107,Poolife Chlorine & Sanitizers
9,Sirona Brom Tabs | 82235,effective and continuous sanitization | powerf...,$42.99,In stock,The NEW Sirona Brom Tabs areslow-dissolving br...,https://www.epoolsupply.com/collections/sirona...,82235,Sirona Chlorine & Sanitizers


In [15]:
df.to_csv('epoolsupply_chlorine.csv', index=False)

In [16]:
def print_row(df, row_index):
    if row_index < 0 or row_index >= len(df):
        print(f"Row index {row_index} is out of bounds (0 - {len(df)-1})")
        return

    row = df.iloc[row_index]

    print(f"\n--- Row {row_index} ---")
    for col in df.columns:
        value = str(row[col])  # Ensure it's a string for splitting
        print(f"{col}:")
        # Split on \n and print each line with indentation
        for line in value.split('\n'):
            print(f"  {line}")
        print("-" * 40)

# Example usage:
# df = pd.read_csv('your_file.csv')
# print_row(df, 2)  # View row at index 2


In [18]:
import pandas as pd 
df = pd.read_csv('epoolsupply_chlorine.csv')

In [23]:
print_row(df, 0)


--- Row 0 ---
Title:
  Poolife 3" Chlorine Tabs (50lbs) | 42118
----------------------------------------
Overview:
  steady, consistent chlorination | sanitized, clear, and safe | slow-dissolving, stabilized tablets | automatic feeders, skimmers, or floating dispensers | consistent | minimizing maintenance efforts | Key Features: | Usage Instructions: | Important Notes: | Benefits: | Poolife 3" Chlorine Tabs (50 LBS) | powerful, efficient, and reliable solution | consistent, long-term chlorination
----------------------------------------
Price:
  $169.95
----------------------------------------
Stock:
  In stock
----------------------------------------
Description:
  Poolife 3" Chlorine Tabs arehighly effective chlorine tabletsthat providesteady, consistent chlorinationto keep your pool watersanitized, clear, and safefor swimming. Theseslow-dissolving, stabilized tabletsare designed forautomatic feeders, skimmers, or floating dispensers, making them a convenient and reliable solution 

In [22]:
df

,Title,Overview,Price,Stock,Description,Link,Manufacturer SKU,Brand
0,"Poolife 3"" Chlorine Tabs (50lbs) | 42118","steady, consistent chlorination | sanitized, c...",$169.95,In stock,"Poolife 3"" Chlorine Tabs arehighly effective c...",https://www.epoolsupply.com/collections/poolif...,42118,Poolife Chlorine & Sanitizers
1,"Poolife 1"" Cleaning Tablets | 42104 (5LBS)",pool maintenance | 1-inch trichlor tablets | k...,$52.99,In stock,"Poolife 1"" Cleaning Tablets arehighly effectiv...",https://www.epoolsupply.com/collections/poolif...,42104,Poolife Chlorine & Sanitizers
2,Poolife Bromine Tablets | 25 LBS (62182),slow-dissolving tablets | continuous disinfect...,$299.00,Out of stock,Key Features:,https://www.epoolsupply.com/collections/poolif...,62182,Poolife Chlorine & Sanitizers
3,Poolife NST Prime Tablets | 4.5 LBS,chlorine | oxidizers | stabilizers | residenti...,$41.99,Out of stock,Poolife NST Prime Tablets are designed forpool...,https://www.epoolsupply.com/collections/poolif...,22427,Poolife Chlorine & Sanitizers
4,Poolife NST Prime Tablets | 9 LBS (22424),slow-dissolving tablets | chlorine sanitizers ...,$72.99,Out of stock,Poolife NST Prime Tablets in the9 LBSsize are ...,https://www.epoolsupply.com/collections/poolif...,22424,Poolife Chlorine & Sanitizers
5,Poolife NST Prime Tablets | 20.2 LBS (22425),larger pools | chlorine | oxidizing | long-las...,$124.99,Out of stock,Poolife NST Prime Tablets in the20.2 LBSsize a...,https://www.epoolsupply.com/collections/poolif...,22425,Poolife Chlorine & Sanitizers
6,Poolife NST Prime Tablets | 39.4 LBS (22426),39.4 LBS | dual-action sanitization | oxidatio...,$276.99,In stock,ThePoolife NST Prime Tabletsin the39.4 LBSsize...,https://www.epoolsupply.com/collections/poolif...,22426,Poolife Chlorine & Sanitizers
7,"Poolife 3"" Cleaning Tablet | 1, 7 OZ (42130)",sanitized and crystal clear | slow-dissolving ...,$6.99,In stock,"Poolife 3"" Cleaning Tablets providelong-lastin...",https://www.epoolsupply.com/collections/poolif...,42130,Poolife Chlorine & Sanitizers
8,"Poolife 3"" Cleaning Tablets | 4.81 LBS (42107)",sanitized and crystal clear | slow-dissolving ...,$42.99,Out of stock,"Poolife 3"" Cleaning Tablets providelong-lastin...",https://www.epoolsupply.com/collections/poolif...,42107,Poolife Chlorine & Sanitizers
9,Sirona Brom Tabs | 82235,effective and continuous sanitization | powerf...,$42.99,In stock,The NEW Sirona Brom Tabs areslow-dissolving br...,https://www.epoolsupply.com/collections/sirona...,82235,Sirona Chlorine & Sanitizers
